In [1]:
# Useful starting lines
%matplotlib inline
import datetime
import costs
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload

%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [ ]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)


In [ ]:
tX_std = standardize(tX)
print(tX_std.shape)

# least squares GD

In [47]:
from gradient_descent import *
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    return gradient_descent(y, tx, initial_w, max_iters, gamma)

# least square SGD

In [48]:
from stochastic_gradient_descent import *
def least_squares_SGD(y, tx, initial_w, batch_size, max_iters, gamma):
    return stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma)

In [49]:
from sklearn.metrics import mean_squared_error

# test GD

In [50]:
# from gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.1
# Initialization
w_initial = np.zeros(30)

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = least_squares_GD(y, tX_std, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

ValueError: shapes (30,) and (250000,) not aligned: 30 (dim 0) != 250000 (dim 0)

In [26]:
standardize(tX)[0]

array([[ 0.72343805,  0.52317289,  0.62968263, ...,  0.40687538,
         0.3983056 ,  0.66583024],
       [ 0.77526501,  0.56264922,  0.64215782, ..., -1.90048197,
        -1.90048197,  0.51064925],
       [-1.90048197,  0.77811391,  0.69456379, ..., -1.90048197,
        -1.90048197,  0.50609331],
       ...,
       [ 0.64728354,  0.54363654,  0.57896063, ..., -1.90048197,
        -1.90048197,  0.50088224],
       [ 0.62304826,  0.44867927,  0.56275072, ..., -1.90048197,
        -1.90048197,  0.40401494],
       [-1.90048197,  0.57184875,  0.56740816, ..., -1.90048197,
        -1.90048197,  0.40401494]])

# test SGD

In [122]:
from stochastic_gradient_descent import *
from proj1_helpers import batch_iter
# Define the parameters of the algorithm.
max_iters = 500
gamma = 0.05
batch_size = 10000

# Initialization
w_initial = np.zeros(30)
print(w_initial)

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = least_squares_SGD(
    y, tX, w_initial,batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))



[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
stoch Gradient Descent(0/499): loss=1.0
stoch Gradient Descent(1/499): loss=0.8748329763372845
stoch Gradient Descent(2/499): loss=0.8750850686707375
stoch Gradient Descent(3/499): loss=0.8686686036034618
stoch Gradient Descent(4/499): loss=0.8608679148802753
stoch Gradient Descent(5/499): loss=0.8607693417990282
stoch Gradient Descent(6/499): loss=0.8645060672515698
stoch Gradient Descent(7/499): loss=0.8604048066595569
stoch Gradient Descent(8/499): loss=0.8532792426369813
stoch Gradient Descent(9/499): loss=0.8575215281097782
stoch Gradient Descent(10/499): loss=0.8485336421575519
stoch Gradient Descent(11/499): loss=0.8520513621131535
stoch Gradient Descent(12/499): loss=0.8395967875366114
stoch Gradient Descent(13/499): loss=0.8488755017591948
stoch Gradient Descent(14/499): loss=0.8539659247005784
stoch Gradient Descent(15/499): loss=0.8450843544715863
stoch Gradient Descent(16/499): loss

stoch Gradient Descent(147/499): loss=0.8061374915319588
stoch Gradient Descent(148/499): loss=0.8057495594774132
stoch Gradient Descent(149/499): loss=0.8087177981046925
stoch Gradient Descent(150/499): loss=0.7982180081773258
stoch Gradient Descent(151/499): loss=0.7984030151537261
stoch Gradient Descent(152/499): loss=0.8011568911031891
stoch Gradient Descent(153/499): loss=0.8063763864696846
stoch Gradient Descent(154/499): loss=0.8061577064439022
stoch Gradient Descent(155/499): loss=0.7963395454379539
stoch Gradient Descent(156/499): loss=0.7966018660329781
stoch Gradient Descent(157/499): loss=0.806803437401804
stoch Gradient Descent(158/499): loss=0.8118941246472636
stoch Gradient Descent(159/499): loss=0.7947371277242355
stoch Gradient Descent(160/499): loss=0.8046814947530171
stoch Gradient Descent(161/499): loss=0.8048720667967942
stoch Gradient Descent(162/499): loss=0.8090859474777933
stoch Gradient Descent(163/499): loss=0.794012111703444
stoch Gradient Descent(164/499): 

stoch Gradient Descent(291/499): loss=0.7918754921128127
stoch Gradient Descent(292/499): loss=0.7947102942921181
stoch Gradient Descent(293/499): loss=0.7970729833823954
stoch Gradient Descent(294/499): loss=0.7925503747669687
stoch Gradient Descent(295/499): loss=0.7964708537209197
stoch Gradient Descent(296/499): loss=0.7882957711192141
stoch Gradient Descent(297/499): loss=0.7973246855470593
stoch Gradient Descent(298/499): loss=0.7932278908118691
stoch Gradient Descent(299/499): loss=0.7949484684293783
stoch Gradient Descent(300/499): loss=0.7970251341977042
stoch Gradient Descent(301/499): loss=0.7916818818127959
stoch Gradient Descent(302/499): loss=0.7938200634785672
stoch Gradient Descent(303/499): loss=0.8042259855063839
stoch Gradient Descent(304/499): loss=0.7922766011903123
stoch Gradient Descent(305/499): loss=0.8007922907847418
stoch Gradient Descent(306/499): loss=0.8058251280523876
stoch Gradient Descent(307/499): loss=0.7963160078009652
stoch Gradient Descent(308/499)

stoch Gradient Descent(437/499): loss=0.7900002671444093
stoch Gradient Descent(438/499): loss=0.7898304897120153
stoch Gradient Descent(439/499): loss=0.7900841648543794
stoch Gradient Descent(440/499): loss=0.7878673670731702
stoch Gradient Descent(441/499): loss=0.7811744261804803
stoch Gradient Descent(442/499): loss=0.7933050045201502
stoch Gradient Descent(443/499): loss=0.7809315712559937
stoch Gradient Descent(444/499): loss=0.7935161965999538
stoch Gradient Descent(445/499): loss=0.784346073610442
stoch Gradient Descent(446/499): loss=0.7986953618346734
stoch Gradient Descent(447/499): loss=0.7823977919892249
stoch Gradient Descent(448/499): loss=0.7913061202313116
stoch Gradient Descent(449/499): loss=0.7857303057764536
stoch Gradient Descent(450/499): loss=0.7955469524389579
stoch Gradient Descent(451/499): loss=0.7974847871216173
stoch Gradient Descent(452/499): loss=0.7899281461620575
stoch Gradient Descent(453/499): loss=0.7913736624724342
stoch Gradient Descent(454/499):

# Least square stochastic gradient descent    

In [65]:
# from stochastic_gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.5
batch_size = 1

# Initialization
w_initial = np.zeros(30)
print(w_initial)

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = least_squares_GD(
    y, tX, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))



[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Gradient Descent(0/49): loss=1.0, w0=53.83324755201073, w1=-13.642099301999938
Gradient Descent(1/49): loss=1004083277570.4924, w0=-47840286.291933425, w1=22451160.509754848
Gradient Descent(2/49): loss=8.251913580469985e+24, w0=127768620470832.92, w1=-64097460738762.164
Gradient Descent(3/49): loss=6.784291354610181e+37, w0=-3.6532296986934156e+20, w1=1.8377016000335243e+20
Gradient Descent(4/49): loss=5.5776973393808864e+50, w0=1.047340667076021e+27, w1=-5.269255590689982e+26
Gradient Descent(5/49): loss=4.5856975017480705e+63, w0=-3.0030238171911826e+33, w1=1.5108608669042093e+33
Gradient Descent(6/49): loss=3.7701259676790215e+76, w0=8.610604774822291e+39, w1=-4.332112251771585e+39
Gradient Descent(7/49): loss=3.099604761708956e+89, w0=-2.468930447217248e+46, w1=1.242152536350795e+46
Gradient Descent(8/49): loss=2.548336517446234e+102, w0=7.079198370712734e+52, w1=-3.5616411438332573e+52
Gr

/Users/Maxime/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in square
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Maxime/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)


Gradient Descent(26/49): loss=inf, w0=1.2148934941648552e+169, w1=-6.112294683573026e+168
Gradient Descent(27/49): loss=inf, w0=-3.483480915230666e+175, w1=1.7525867066338204e+175
Gradient Descent(28/49): loss=inf, w0=9.988232997426503e+181, w1=-5.025216098504703e+181
Gradient Descent(29/49): loss=inf, w0=-2.863939858969307e+188, w1=1.4408871607370347e+188
Gradient Descent(30/49): loss=inf, w0=8.211814359863811e+194, w1=-4.1314756804082574e+194
Gradient Descent(31/49): loss=inf, w0=-2.3545848866090442e+201, w1=1.184623734801979e+201
Gradient Descent(32/49): loss=inf, w0=6.75133380431142e+207, w1=-3.39668801564262e+207
Gradient Descent(33/49): loss=inf, w0=-1.9358192773793115e+214, w1=9.739370516275073e+213
Gradient Descent(34/49): loss=inf, w0=5.5506013823228e+220, w1=-2.7925831756244726e+220
Gradient Descent(35/49): loss=inf, w0=-1.5915316096631008e+227, w1=8.007212354996864e+226
Gradient Descent(36/49): loss=inf, w0=4.56342059190862e+233, w1=-2.295919070832187e+233
Gradient Descent(3

In [35]:
# Time Visualization
from plots import gradient_descent_visualization

from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        sgd_losses, sgd_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(sgd_ws)))



ModuleNotFoundError: No module named 'plots'

## Do your thing crazy machine learning thing here :) ...

## Generate predictions and save ouput in csv format for submission:

In [ ]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)